In [1]:
import pandas as pd
from transformers import pipeline


df = pd.read_csv('raw_data/final_table.csv')
df.head(2)

2024-03-10 08:28:25.785303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,title,paths,imdb_id_x,plot_synopsis,tags,split,synopsis_source,imdb_id_y,synopsis,genre
0,12 monkeys,data/resized_frames/12 Monkeys (1995),tt3148266,A deadly virus released in 1996 wipes out almo...,murder,train,wikipedia,tt75933,"In a future world devastated by disease, a con...",scifi
1,21 grams,data/resized_frames/21 Grams (2003),tt0315733,The story is told in a non-linear manner. The ...,"suspenseful, boring, murder, realism, depressi...",test,wikipedia,tt18010,A freak accident brings together a critically ...,crime


In [2]:
df['plot_synopsis'].head()	

0    A deadly virus released in 1996 wipes out almo...
1    The story is told in a non-linear manner. The ...
2    Late one evening, at the U.S. Naval Base in Gu...
3    Leland Jones and William 'Billy' Orser (Stephe...
4    Leland Jones and William 'Billy' Orser (Stephe...
Name: plot_synopsis, dtype: object

In [4]:
  import pandas as pd
  from transformers import pipeline
  from tqdm import tqdm

  # Initialize the summarization pipeline with the model
  summarizer = pipeline("summarization", model="Falconsai/text_summarization")

  def summarize_column(df, column_name, num_rows=2, save_path=None, run_interval=20):
      # Function to summarize each row in the specified column
      def summarize_text(text):
          summary = summarizer(text, max_length=150, min_length=30, do_sample=False)
          return summary[0]['summary_text']

      # Initialize a counter for tracking runs
      run_count = 0
      summarized_data = []

      # Load previously saved data if it exists
      try:
          if save_path:
              summarized_data = pd.read_csv(save_path)
              run_count = len(summarized_data)
      except FileNotFoundError:
          pass

      # Check if the current run is one of the saving intervals
      if run_count % run_interval == 0 or not summarized_data:
          df_subset = df.head(num_rows)  # Select the specified number of rows from the DataFrame

          # Apply the summarization function to the selected rows
          tqdm.pandas(desc="Summarizing column")  # Set up tqdm to show progress
          df_subset['Summarized_' + column_name] = df_subset[column_name].progress_apply(summarize_text)

          # Concatenate new summarized data with old ones
          summarized_data = pd.concat([summarized_data, df_subset], ignore_index=True)

          # Save the updated DataFrame to a CSV file every 20 runs or initially
          if save_path:
              summarized_data.to_csv(save_path, index=False)

      return summarized_data.tail(num_rows)  # Return the last `num_rows` summarized items

  # Example usage:
  # Assuming df is your DataFrame and 'synopsis' is the column you want to summarize
  save_path = '/content/sample_data/partially_processed_data.csv'
  df_selected = summarize_column(df, 'plot_synopsis	', num_rows=4, save_path=save_path)
  print(df_selected)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


KeyError: 'plot_synopsis\t'

In [5]:
rows = len(df['plot_synopsis'])
rows

425

In [6]:
num_rows = len(df['plot_synopsis'].keys)

TypeError: object of type 'method' has no len()

In [7]:
df_selected = summarize_column(df, 'plot_synopsis', num_rows=rows)

Summarizing column:   0%|                               | 0/425 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (871 > 512). Running this sequence through the model will result in indexing errors
2024-03-10 08:30:50.644621: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8e0c9ae700 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-10 08:30:50.644640: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-03-10 08:30:50.644951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 08:30:50.649638: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling M

KeyboardInterrupt: 

In [ ]:
df_selected['Summarized_plot_synopsis']